### Boiler Plate

It's a standard practice to start jupyter notebook with the following three lines; they ensure that any edits to libraries you make are reloaded here automatically, and also that any charts or images displayed are shown in this notebook.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Importing Fast AI library

Let's import fastai library and define our batch_size parameter to 64. Usually, image databases are huge, so we need to feed these images into a GPU using batches, batch size 64 means that we will feed 64 images at once to update parameters of our deep learning model. If you are running out of memory because of smaller GPU RAM, you can reduce batch size to 32 or 16.

In [ ]:
from fastai import *
from fastai.vision import *

bs = 64

### Looking at the data

The first thing we do when we approach a problem is to take a look at the data. We always need to understand very well what the problem is and what the data looks like before we can figure out how to solve it. Taking a look at the data means understanding how the data directories are structured, what the labels are and what some sample images look like. Our data is already split in train and validation folder, and inside each subdirectory, our folder name represents the class name of all the images present within that subfolder. Fortunately, the fastai library has a handy function made exactly for this, [ImageDataBunch.from_folder](https://docs.fast.ai/vision.data#ImageDataBunch.from_folder) gets the label names from the folder name automatically. fastai library have an [awesome documentation](https://docs.fast.ai/) to navigate through their library functions with live examples on how to use them. Once the data is loaded, we can also normalize the data by using .normalize to ImageNet parameters.

In [ ]:
## Declaring path of dataset
path_img = Path('../../data/malaria/')

## Loading data 
# data = ImageDataBunch.from_folder(path=path_img, train='train', valid='valid', ds_tfms=get_transforms(),size=224, bs=bs, check_ext=False)
data = ImageDataBunch.from_folder(path=path_img, train='train', valid='valid', size=224, bs=bs, check_ext=False)

## Normalizing data based on Image net parameters
data.normalize(imagenet_stats)

To look at a random sample of images, we can use .show_batch() function ImageDataBunch class. As we can see below we have some cases of diseases leaf on different crops plus some background noise images from DAGS dataset which will act as noise.

In [ ]:
data.show_batch(rows=3, figsize=(10,8))

Let's print all the data classes present in the database. In total, we have images in 3 classes as mentioned above in the motivation section.

In [ ]:
print(data.classes)
len(data.classes),data.c

## Transfer learning using a pretrained model : ResNet 50

Now we will start training our model. We will use a convolutional neural network backbone [ResNet 50](https://medium.com/@14prakash/understanding-and-implementing-architectures-of-resnet-and-resnext-for-state-of-the-art-image-cf51669e1624) and a fully connected head with a single hidden layer as a classifier. You can also read the [ResNet paper](https://arxiv.org/pdf/1512.03385.pdf) if you want to understand all the architecture detail. To create the transfer learing model we will need to use function create_cnn from Learner class and feed a pretrained model from models class.

In [ ]:
## To create a ResNET 50 with pretrained weights
learn = create_cnn(data, models.resnet50, metrics=error_rate)

The ResNet50 model created by create_cnn function have initial layers frozen and we are just going to learn weights of the last fully connected layers.

In [ ]:
learn.fit_one_cycle(5)

In [ ]:
learn.save('model_stage1')

FastAI library also provides functions to explore results faster and find if our model is learning what it is supposed to learn.
We will first see which were the categories that the model most confused with one another. We will try to see if what the model predicted was reasonable or not using ClassificationInterpretation class.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(4, figsize=(20,25))

In this case, the model is getting confused in detecting Northern Leaf blight from Gray leaf spots on Corn plant and early/late blight in tomato leaves which visually looks pretty similar. This is an indicator that our classifier is working correctly.
Furthermore, when we plot the confusion matrix, we can see that most of the things are classified correctly, and it's almost a near perfect model.

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=60)

So until now we have only been training the last classification layers, but what if we want to optimize earlier layers too. In transfer learning playing with earlier layers should be done with caution and the learning rate should be kept pretty low. FastAI library provides a function to see what will be the ideal learning rate to train upon, so let's plot it. The lr_find function runs the model for a subset of data at multiple learning rate to determine which learning rate would be best.

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

It looks like we should keep our learning rate lower than 10e-4. We can use slice function to logarithmically distribute learning rate between 10e-6 to 10e-4 for different layers in the network. Keeping the lowest learning rate for the initial layers and increasing it for later layers. Let's unfreeze all the layers so that we can train the entire model using unfreeze() function.

In [ ]:
learn.load('model_stage1')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-7,1e-5))